In [1]:
import os
import pandas as pd
import numpy as np
import folium

from tqdm.auto import tqdm
from folium import plugins

In [2]:
gps_data_dir = 'E:/data/gps'
road_data_dir = 'E:/data/road'

In [3]:
tqdm.pandas(desc='working : ')

# Explore gps data

## find interesting road for use in project

### load GPS data
- **Columns**
  - time_stamp => เป็นวันเวลา
  - unit_id => เป็น id ของรถ
  - lat => เป็นค่า latitude
  - lon => เป็นค่า longitude
  - speed => เป็นค่าความเร็ว หน่วยเป็น km/h
  - unit_type => เป็นชนิดของรถ
    - 1 = รถโดยสารประจำทาง
    - 3 = รถโดยสารไม่ประจำทาง
    - 4 = รถโดยสารส่วนบุคคล
    - 5,8,9  = รถบรรทุกส่วนบุคคล
    - 6,7 = รถบรรทุกไม่ประจำทาง

In [18]:
# df_gps = pd.read_csv(gps_data_dir + '/2019/2019-03/2019-03-01_00.zip')
# df_gps.head()

,time_stamp,unit_id,lat,lon,speed,unit_type
0,2019-03-01 00:00:35,0390002000000000000EE085491,13.702763,100.581581,2,7.0
1,2019-03-01 00:00:26,015000500000863835028323652,9.970495,98.642845,0,3.0
2,2019-03-01 00:00:32,0390002000000000000EE107549,19.950153,99.236827,3,8.0
3,2019-03-01 00:00:28,005000600000864507030191018,14.076360,100.520670,0,8.0
4,2019-03-01 00:00:33,025000600000359857080832934,12.516950,99.978370,0,7.0


### load road data
- **Columns**
  - rid => หมายเลขสายถนน
  - rd => เลขถนน
  - km
  - ptype => ชนิดหลัก
  - lat, lon => ข้อมูลพิกัดแบบละเอียด
  - latx, lonx => ข้อมูลพิกัด ทศนิยม 3 ตำแหน่ง
  - angle => องศาของถนน 

In [4]:
# df_road = pd.read_csv(road_data_dir + '/roaddb.csv')
# df_road.head()

,rid,rd,km,ptype,lat,lon,latx,lonx,angle
0,1,3003,0.0,100m,14.492856,100.066108,14.493,100.066,180
1,1,3003,0.1,100m,14.493738,100.066290,14.494,100.066,180
2,1,3003,0.2,100m,14.494619,100.066476,14.495,100.066,179
3,1,3003,0.3,100m,14.495501,100.066658,14.496,100.067,179
4,1,3003,0.4,100m,14.496383,100.066839,14.496,100.067,179


### find interesting road

In [12]:
def get_road_dict(df_road):
    road_list = []
    df_road['coor'] = df_road.progress_apply(lambda row: f"{row['latx']:.3f}, {row['lonx']:.3f}", axis='columns')
    for rd in df_road['rd'].unique():
        coors = df_road[df_road['rd'] == rd]['coor'].to_list()
        road_dict = {
            'rd': rd,
            'coor_list': coors,
        }
        road_list.append(road_dict)
    return road_list

def get_road_interest(df_gps, road_list):
    df_gps = df_gps[df_gps['unit_type'].isin([6, 7])].reset_index(drop=True)
    df_gps = df_gps.drop_duplicates(subset=['unit_id']).reset_index(drop=True)
    df_gps['coor'] = df_gps.progress_apply(lambda row: f"{row['lat']:.3f}, {row['lon']:.3f}", axis='columns')
    for idx in range(len(road_list)):
        count_car = len(df_gps[df_gps['coor'].isin(road_list[idx]['coor_list'])])
        road_list[idx]['count_car'] = count_car

    return road_list

def find_road_interest():
    df_road = pd.read_csv(road_data_dir + '/roaddb.csv')
    df_gps = pd.read_csv(gps_data_dir + '/2019/2019-03/2019-03-01_00.zip')

    road_list = get_road_dict(df_road)
    road_list = get_road_interest(df_gps, road_list)

    # sort 'count_car' from highest to lowest.
    road_list.sort(key=sort_count, reverse=True)

    # clear data in memory
    del df_road
    del df_gps

    return road_list

def sort_count(d):
    """
    function get value for sort list
    """
    return d['count_car']

In [13]:
road_list = find_road_interest()

working : 100%|██████████| 97254/97254 [00:01<00:00, 93987.05it/s]


In [14]:
# show top 10 road number have trucks
for idx in range(10):
    print(f"{road_list[idx]['rd']} : {road_list[idx]['count_car']}")

4 : 2568
2 : 2267
1 : 2226
41 : 1022
7 : 974
9 : 946
304 : 778
32 : 726
3902 : 718
3901 : 717
